In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import osm2geojson
import json
import polars as pl
import pyarrow
import contextily as cx
import os
from local_functions import *

pl.Config.set_tbl_rows(200)
pl.Config.set_fmt_float("full")

polars.config.Config

In [3]:
territory_population_full = (
    pd.read_csv("data/Iedzīvotāju skaits.csv", sep=";", encoding="utf-8", na_values=["…"])
    .rename(columns={"Laika periods": "Gads"})
)
territory_population_full["Teritoriālā vienība"] = territory_population_full["Teritoriālā vienība"].str.replace(" novads", " nov.")

def get_territory_population(year):
    return (
        territory_population_full[territory_population_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [4]:
territory_population_salary_full = (
    pd.read_csv("data/RIG110_Mēneša darba samaksa.csv", sep=";", encoding="utf-8")
    .drop("Bruto/ Neto", axis=1)
    .rename(columns={"Visas teritorijas": "Teritoriālā vienība", "Laika periods": "Gads"})
)
territory_population_salary_full["Teritoriālā vienība"] = territory_population_salary_full["Teritoriālā vienība"].str.slice(10).str.replace(" novads", " nov.")

def get_territory_population_salary(year):
    return (
        territory_population_salary_full[territory_population_salary_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [5]:
territory_workable_population_full = (
    pd.read_csv("data/IRD081_Iedzīvotāji pēc dzimuma un vecuma grupām.csv", sep=";", encoding="utf-8")
    .drop(["Dzimums", "Vecuma grupa"], axis=1)
    .rename(columns={"Laika periods": "Gads"})
)
territory_workable_population_full["Teritoriālā vienība"] = territory_workable_population_full["Teritoriālā vienība"].str.slice(10).str.replace(" novads", " nov.")


def get_territory_workable_population(year):
    return (
        territory_workable_population_full[territory_workable_population_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [6]:
territory_population_with_higher_education_full = (
    pd.read_csv("data/RIG070_iedzīvotāju īpatsvars ar augstāko izglītību.csv", sep=";", encoding="utf-8")
    .rename(columns={"Laika periods": "Gads", "Visas teritorijas": "Teritoriālā vienība"})
)

territory_population_with_higher_education_full["Teritoriālā vienība"] = territory_population_with_higher_education_full["Teritoriālā vienība"].str.slice(10).str.replace(" novads", " nov.")

def get_higher_education_proportion(year):
    return (
        territory_population_with_higher_education_full[territory_population_with_higher_education_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [7]:
gdp_full = (
    pd.read_csv("data/IKR061_Iekšzemes kopprodukts un bruto pievienotā vērtība.csv", sep=";", encoding="utf-8")
    .rename(columns={"Laika periods": "Gads"})
)
gdp_full["Teritoriālā vienība"] = gdp_full["Teritoriālā vienība"].str.replace(" novads", " nov.")

def get_gdp(year):
    return (
        gdp_full[gdp_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [8]:
territory_population_economic_activity_full = (
    pd.read_csv("data/EKA140_Ekonomiskās aktivitātes, nodarbinātības un bezdarba līmenis.csv", sep=";", encoding="utf-8")
    .rename(columns={"Laika periods": "Gads"})
    .drop(["Dzimums", "Vecums"], axis=1)
)
territory_population_economic_activity_full["Teritoriālā vienība"] = territory_population_economic_activity_full["Teritoriālā vienība"].str.replace(" novads", " nov.")

def get_territory_population_economic_activity(year):
    return (
        territory_population_economic_activity_full[territory_population_economic_activity_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [9]:
territory_companies_full = (
    pd.read_csv("data/UZS041_Tirgus sektora ekonomiski aktīvi uzņēmumi reģionos.csv", sep=";", encoding="utf-8")
    .rename(columns={"Laika periods": "Gads"})
)

territory_companies_full["Teritoriālā vienība"] = territory_companies_full["Teritoriālā vienība"].str.replace(" novads", " nov.")


def get_territory_companies(year):
    return (
        territory_companies_full[territory_companies_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [10]:
product_value_full = (
    pd.read_csv("data/Radītā pievienotā un produkcijas vērtība.csv", sep=";", encoding="utf-8")
    .rename(columns={"Laika periods": "Gads"})
)

product_value_full["Teritoriālā vienība"] = product_value_full["Teritoriālā vienība"].str.slice(10).str.replace(" novads", " nov.")

def get_product_value(year):
    return (
        product_value_full[product_value_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [11]:
territory_water_usage_full = (
    pd.read_csv("data/DRU020_Ūdens izmantošana.csv", sep=";", encoding="utf-8")
    .rename(columns={"Laika periods": "Gads", "Rādītāji": "Teritoriālā vienība"})
)

territory_water_usage_full["Teritoriālā vienība"] = territory_water_usage_full["Teritoriālā vienība"].str.replace(" novads", " nov.")

def get_territory_water_usage(year):
    return (
        territory_water_usage_full[territory_water_usage_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [12]:
territory_road_financing_full = (
    pd.read_csv("data/TRI011_ceļu un ielu finansēšanas līdzekļu izmantošana.csv", sep=";", encoding="utf-8", na_values=["-"])
    .rename(columns={"Laika periods": "Gads"})
)

territory_road_financing_full["Teritoriālā vienība"] = territory_road_financing_full["Teritoriālā vienība"].str.replace(" novads", " nov.")

def get_territory_road_financing(year):
    return (
        territory_road_financing_full[territory_road_financing_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [13]:
road_length_full = (
    pd.read_csv("data/TRS020_autoceļu un pašvaldību ielu garums.csv", sep=";", encoding="utf-8")
    .rename(columns={"Laika periods": "Gads"})
)

road_length_full["Teritoriālā vienība"] = road_length_full["Teritoriālā vienība"].str.replace(" novads", " nov.")

def get_road_length(year):
    return (
        road_length_full[road_length_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [14]:
social_spending_full = (
    pd.read_csv("data/PPS060_izdevumi sociālā atbalsta.csv", sep=";", encoding="utf-8")
    .rename(columns={"Laika periods": "Gads"})
)

social_spending_full["Teritoriālā vienība"] = social_spending_full["Teritoriālā vienība"].str.replace(" novads", " nov.")

def get_social_spending(year):
    return (
        social_spending_full[social_spending_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [15]:
new_apartment_area_full = (
    pd.read_csv("data/BUE0211_Ekspluatācijā pieņemto jauno dzīvokļu platība reģionos.csv", sep=";", encoding="utf-8", na_values=["-"])
    .rename(columns={"Laika periods": "Gads"})
)

new_apartment_area_full["Teritoriālā vienība"] = new_apartment_area_full["Teritoriālā vienība"].str.replace(" novads", " nov.")

def get_new_apartment_area(year):
    return (
        new_apartment_area_full[new_apartment_area_full["Gads"] == year]
        .drop("Gads", axis=1)
    )

In [16]:
functional_classifier = pl.read_excel("data/pasvaldibu_izdevumu_funkcionala_klasifikacija.xlsx")

def get_region_government_expenses(year):
    if os.path.exists(f"data/pasvaldibu_{year}_izdevumi.xlsx"):
        file_exists = True
        file_path = f"data/pasvaldibu_{year}_izdevumi.xlsx"
    else:
        file_exists = False
        file_path = "data/pasvaldibu_2023_izdevumi.xlsx"
    
    return (
        pl.read_excel(file_path)
        .with_columns(
            pl.col("Funkcionalas_klasifikacijas_kods").cast(pl.Float64),
            pl.col("Pasvaldibas_nosaukums").str.replace_all("-- ", "").str.replace_all("- ", "")
        )
        .join(
            functional_classifier,
            how="left",
            left_on="Funkcionalas_klasifikacijas_kods",
            right_on="Kods"
        )
        .with_columns(
            pl.col("Koda nosaukums").alias("Funkcionala_klasifikacija")
        )
        .filter(
            pl.col("Izdevumu_ekonomiskas_klasifikacijas_kods").is_in(["1.0.", "2.0."])
        )
        .with_columns(
            pl.col("Izdevumu_ekonomiskas_klasifikacijas_kods").replace({"1.0.": "Uzturēšanas izdevumi", "2.0.": "Kapitālie izdevumi" }).alias("Izdevumu_ekonomiska_klasifikacija")
        )
        .with_columns(
            ("Pašvaldības izdevumi: " + pl.col("Funkcionala_klasifikacija") + " (" + pl.col("Izdevumu_ekonomiska_klasifikacija") + ")").alias("Izdevumu tips")
        )
        .pivot(
            on="Izdevumu tips",
            index=["Pasvaldibas_nosaukums"],
            values=["EUR"],
            aggregate_function="sum"
        )
        .with_columns(
            pl.col("Pasvaldibas_nosaukums").str.replace(" novads", " nov.")
        )
        .rename({"Pasvaldibas_nosaukums": "Teritoriālā vienība"})
        .filter(
            pl.lit(file_exists)
        )
        .to_pandas()
    )

In [17]:
income_functional_classifier = pl.read_excel("data/pasvaldibu_ienemumu_funkcionala_klasifikacija.xlsx")

In [18]:
def get_region_government_income(year):
    if os.path.exists(f"data/pasvaldibu_{year}_ienemumi.xlsx"):
        file_exists = True
        file_path = f"data/pasvaldibu_{year}_ienemumi.xlsx"
    else:
        file_exists = False
        file_path = "data/pasvaldibu_2023_ienemumi.xlsx"
    
    return (
        pl.read_excel(file_path)
        .with_columns(
            pl.col("Pasvaldibas nosaukums").str.replace_all("-- ", "").str.replace_all("- ", "")
        )
        .filter(
            pl.col("Ienemumu_klasifikacijas_kods").is_in(["1.1.0.0.", "1.2.0.0.", "2.1.0.0.", "2.2.0.0.", "2.5.0.0.", "4.1.0.0.", "5.1.0.0.", "5.2.0.0.", "5.3.0.0.", "5.4.0.0.", "5.5.0.0.", "6.0.0.0.", "7.2.0.0.", "8.0.0.0.", "9.0.0.0.", "9.3.0.0.", "10.0.0.0."])
        )
        .join(
            income_functional_classifier,
            how="left",
            left_on="Ienemumu_klasifikacijas_kods",
            right_on="Kods"
        )
        .with_columns(
            ("Pašvaldības ieņēmumi: " + pl.col("Koda nosaukums un skaidrojums")).alias("Koda nosaukums un skaidrojums")
        )
        .pivot(
            on="Koda nosaukums un skaidrojums",
            index="Pasvaldibas nosaukums",
            values=["EUR"],
            aggregate_function="sum"
        )
        .with_columns(
            pl.col("Pasvaldibas nosaukums").str.replace(" novads", " nov.")
        )
        .rename({"Pasvaldibas nosaukums": "Teritoriālā vienība"})
        .filter(
            pl.lit(file_exists)
        )
        .to_pandas()
    )

In [19]:
def prepare_residential_details(residential, healthcare, public_transport, schools, shops):
    return (
        residential
        .sjoin_nearest(healthcare, how="left", distance_col="Attālums līdz ārstam (m)")
        [["Gads_left", "id_left", "geometry", "Daudzdzīvokļu ēka", "Savrupmāja", "AvgCoef", "Attālums līdz ārstam (m)"]]
        .rename(columns={"Gads_left":"Gads", "id_left": "id"})
        .sjoin_nearest(healthcare[healthcare.amenity == "hospital"], how="left", distance_col="Attālums līdz slimnīcai (m)")
        [["Gads_left", "id_left", "geometry", "Daudzdzīvokļu ēka", "Savrupmāja", "AvgCoef", "Attālums līdz ārstam (m)", "Attālums līdz slimnīcai (m)"]]
        .rename(columns={"Gads_left":"Gads", "id_left": "id"})
        .sjoin_nearest(healthcare[healthcare.amenity == "clinic"], how="left", distance_col="Attālums līdz poliklīnikai (m)")
        [["Gads_left", "id_left", "geometry", "Daudzdzīvokļu ēka", "Savrupmāja", "AvgCoef", "Attālums līdz ārstam (m)", "Attālums līdz slimnīcai (m)", "Attālums līdz poliklīnikai (m)"]]
        .rename(columns={"Gads_left":"Gads", "id_left": "id"})
        .sjoin_nearest(public_transport, how="left", distance_col="Attālums līdz sabiedriskajam transportam (m)")
        [["Gads_left", "id_left", "geometry", "Daudzdzīvokļu ēka", "Savrupmāja", "AvgCoef", "Attālums līdz ārstam (m)", "Attālums līdz slimnīcai (m)", "Attālums līdz poliklīnikai (m)", "Attālums līdz sabiedriskajam transportam (m)"]]
        .rename(columns={"Gads_left":"Gads", "id_left": "id"})
        .sjoin_nearest(schools[schools.amenity == "school"], how="left", distance_col="Attālums līdz skolai (m)")
        [["Gads_left", "id_left", "geometry", "Daudzdzīvokļu ēka", "Savrupmāja", "AvgCoef", "Attālums līdz ārstam (m)", "Attālums līdz slimnīcai (m)", "Attālums līdz poliklīnikai (m)", "Attālums līdz sabiedriskajam transportam (m)", "Attālums līdz skolai (m)"]]
        .rename(columns={"Gads_left":"Gads", "id_left": "id"})
        .sjoin_nearest(schools[schools.amenity == "kindergarten"], how="left", distance_col="Attālums līdz bērnudārzam (m)")
        [["Gads_left", "id_left", "geometry", "Daudzdzīvokļu ēka", "Savrupmāja", "AvgCoef", "Attālums līdz ārstam (m)", "Attālums līdz slimnīcai (m)", "Attālums līdz poliklīnikai (m)", "Attālums līdz sabiedriskajam transportam (m)", "Attālums līdz skolai (m)", "Attālums līdz bērnudārzam (m)"]]
        .rename(columns={"Gads_left":"Gads", "id_left": "id"})
        .sjoin_nearest(shops, how="left", distance_col="Attālums līdz veikalam (m)")
        [["Gads_left", "id_left", "geometry", "Daudzdzīvokļu ēka", "Savrupmāja", "AvgCoef", "Attālums līdz ārstam (m)", "Attālums līdz slimnīcai (m)", "Attālums līdz poliklīnikai (m)", "Attālums līdz sabiedriskajam transportam (m)", "Attālums līdz skolai (m)", "Attālums līdz bērnudārzam (m)", "Attālums līdz veikalam (m)"]]
        .rename(columns={"Gads_left":"Gads", "id_left": "id"})
    )

In [20]:
def prepare_territory_residential(territory, residential_details):
    return (
        pl.from_pandas(
            territories
            .sjoin(residential_details, how="left", predicate="contains")
            .drop(["geometry"], axis=1)
        )
        .group_by("Teritoriālā vienība")
        .agg(
            pl.col("Daudzdzīvokļu ēka").sum().alias("Daudzdzīvokļu ēku skaits"),
            pl.col("Savrupmāja").sum().alias("Savrupmāju skaits"),
            ((pl.col("Attālums līdz ārstam (m)") * pl.col("AvgCoef")).sum() / pl.col("AvgCoef").sum()).alias("Vidējais attālums līdz ārstam (m)"),
            ((pl.col("Attālums līdz slimnīcai (m)") * pl.col("AvgCoef")).sum() / pl.col("AvgCoef").sum()).alias("Vidējais attālums līdz slimnīcai (m)"),
            ((pl.col("Attālums līdz poliklīnikai (m)") * pl.col("AvgCoef")).sum() / pl.col("AvgCoef").sum()).alias("Vidējais attālums līdz poliklīnikai (m)"),
            ((pl.col("Attālums līdz sabiedriskajam transportam (m)") * pl.col("AvgCoef")).sum() / pl.col("AvgCoef").sum()).alias("Vidējais attālums līdz sabiedriskajam transportam (m)"),
            ((pl.col("Attālums līdz skolai (m)") * pl.col("AvgCoef")).sum() / pl.col("AvgCoef").sum()).alias("Vidējais attālums līdz skolai (m)"),
            ((pl.col("Attālums līdz bērnudārzam (m)") * pl.col("AvgCoef")).sum() / pl.col("AvgCoef").sum()).alias("Vidējais attālums līdz bērnudārzam (m)"),
            ((pl.col("Attālums līdz veikalam (m)") * pl.col("AvgCoef")).sum() / pl.col("AvgCoef").sum()).alias("Vidējais attālums līdz veikalam (m)"),
        )
        .to_pandas()
    )

In [21]:
def prepare_territory_healthcare(territories, healthcare):
    return (
        pl.from_pandas(
            territories
            .sjoin(healthcare, how="left", predicate="contains")
            .drop("geometry", axis=1)
        )
        .group_by("Teritoriālā vienība")
        .agg(
            pl.col("Slimnīca").sum().alias("Slimnīcu skaits"),
            pl.col("Poliklīnika").sum().alias("Poliklīniku skaits"),
            pl.col("Ārsta prakse").sum().alias("Ārstu prakses skaits")
        )
        .to_pandas()
    )

In [22]:
def prepare_territory_schools(territories, schools):
    return (
        pl.from_pandas(
            territories
            .sjoin(schools, how="left", predicate="contains")
            .drop("geometry", axis=1)
        )
        .group_by("Teritoriālā vienība")
        .agg(
            pl.col("Bērnudārzs").sum().alias("Bērnudārzu skaits"),
            pl.col("Skola").sum().alias("Skolu skaits"),
        )
        .to_pandas()
    )

In [23]:
def prepare_territory_public_transports(territories, public_transport):
    return (
        pl.from_pandas(
            territories
            .sjoin(public_transport, how="left", predicate="contains")
            .drop("geometry", axis=1)
        )
        .group_by("Teritoriālā vienība")
        .agg(
            pl.col("Teritoriālā vienība").count().alias("Sabiedriskā transporta pieturu skaits")
        )
        .to_pandas()
    )

In [24]:
def prepare_territory_shops(territories, shops):
    return (
        pl.from_pandas(
            territories
            .sjoin(shops, how="left", predicate="contains")
            .drop("geometry", axis=1)
        )
        .group_by("Teritoriālā vienība")
        .agg(
            ((pl.col("shop") == "convenience").sum() + (pl.col("shop") == "general").sum()).alias("Veikalu skaits"),
            (pl.col("shop") == "mall").sum().alias("Tirdzniecības centru skaits"),
        )
        .to_pandas()
    )

In [25]:
results = []
years = list(range(2014, 2025))

for year in years:
    print(f"Processing: {year}")
    territory = territories.copy()
    territory["Gads"] = year
    population = get_territory_population(year) 
    population_salary = get_territory_population_salary(year)
    workable_population = get_territory_workable_population(year)
    territory_population_economic_activity = get_territory_population_economic_activity(year)
    territory_companies = get_territory_companies(year)
    water_usage = get_territory_water_usage(year)
    new_apartment_area = get_new_apartment_area(year)
    higher_education_proportion = get_higher_education_proportion(year)
    road_financing = get_territory_road_financing(year)
    social_spending = get_social_spending(year)
    road_length = get_road_length(year)
    gdp = get_gdp(year)
    residential = get_residential(year)
    healthcare = get_healthcare(year)
    public_transport = get_public_transport(year)
    schools = get_schools(year)
    shops = get_shops(year)
    product_value = get_product_value(year)

    gov_expenses = get_region_government_expenses(year)
    gov_income = get_region_government_income(year)

    residential_details = prepare_residential_details(residential, healthcare, public_transport, schools, shops)
    residential_details.to_file(f"./data/transformed/residential_details_{year}.geojson", driver="GeoJSON")
    territory_residential = prepare_territory_residential(territory, residential_details)
    territory_healthcare = prepare_territory_healthcare(territory, healthcare)
    territory_schools = prepare_territory_schools(territory, schools)
    territory_public_transports = prepare_territory_public_transports(territory, public_transport)
    territory_shops = prepare_territory_shops(territory, shops)

    territory_result = (
        territory
        .merge(population, on=["Teritoriālā vienība"], how="left")
        .merge(population_salary, on=["Teritoriālā vienība"], how="left")
        .merge(workable_population, on=["Teritoriālā vienība"], how="left")
        .merge(gov_expenses, on=["Teritoriālā vienība"], how="left")
        .merge(gov_income, on=["Teritoriālā vienība"], how="left")
        .merge(territory_population_economic_activity, on=["Teritoriālā vienība"], how="left")
        .merge(territory_companies, on=["Teritoriālā vienība"], how="left")
        .merge(new_apartment_area, on=["Teritoriālā vienība"], how="left")
        .merge(social_spending, on=["Teritoriālā vienība"], how="left")
        .merge(road_financing, on=["Teritoriālā vienība"], how="left")
        .merge(water_usage, on=["Teritoriālā vienība"], how="left")
        .merge(road_length, on=["Teritoriālā vienība"], how="left")
        .merge(higher_education_proportion, on=["Teritoriālā vienība"], how="left")
        .merge(gdp, on=["Teritoriālā vienība"], how="left")
        .merge(product_value, on=["Teritoriālā vienība"], how="left")
        .merge(territory_residential, on=["Teritoriālā vienība"], how="left")
        .merge(territory_healthcare, on=["Teritoriālā vienība"], how="left")
        .merge(territory_schools, on=["Teritoriālā vienība"], how="left")
        .merge(territory_public_transports, on=["Teritoriālā vienība"], how="left")
        .merge(territory_shops, on=["Teritoriālā vienība"], how="left")
    )

    results.append(territory_result)

Skipping field nodes: unsupported OGR type: 13


Processing: 2014


Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13


Processing: 2015


Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13


...

Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13


Processing: 2023


Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13


Processing: 2024


Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13
Skipping field nodes: unsupported OGR type: 13


In [26]:
territorial_information = pd.concat(results)

In [27]:
for m in territorial_information.columns:
    print(m)

Teritoriālā vienība
Platība (km^2)
geometry
Gads
Iedzīvotāju skaits gada sākumā
Gada vidējais iedzīvotāju skaits
Iedzīvotāju dabiskais pieaugums
Migrācijas saldo
Iedzīvotāju skaita izmaiņas - pavisam
Mēneša darba samaksas mediāna
Iedzīvotāji darbspējas vecumā
Pašvaldības izdevumi: Vispārējie valdības dienesti (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Vispārējie valdības dienesti (Kapitālie izdevumi)
Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Kapitālie izdevumi)
Pašvaldības izdevumi: Ekonomiskā darbība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Kapitālie izdevumi)
Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Kapitālie izdevumi)
Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi)
Pašvaldības izdev

In [28]:
starting_columns = list(territorial_information.columns)
starting_columns.remove('Teritoriālā vienība')
starting_columns.remove('Gads')
starting_columns.remove('geometry')

derived_measures = (
    pl.from_dataframe(territorial_information.loc[:, territorial_information.columns != 'geometry'])
    .with_columns(
        (pl.lit(1.) * pl.col("Iedzīvotāji darbspējas vecumā") / pl.col("Iedzīvotāju skaits gada sākumā")).alias("Iedzīvotāji darbspējas vecumā (%)"),
        (1. * pl.col("Migrācijas saldo") / pl.col("Iedzīvotāju skaits gada sākumā")).alias("Migrācijas saldo (%)"),
        (1. * pl.col("Iedzīvotāju skaita izmaiņas - pavisam") / pl.col("Iedzīvotāju skaits gada sākumā")).alias("Iedzīvotāju skaita izmaiņas - pavisam (%)"),
        *[
            (pl.lit(1.) * pl.col(m) / pl.col("Iedzīvotāju skaits gada sākumā") * 1000).alias(f"{m} (uz 1000 iedz.)")
            for m in [
                "Slimnīcu skaits",
                "Poliklīniku skaits",
                "Ārstu prakses skaits",
                "Bērnudārzu skaits",
                "Skolu skaits",
                "Sabiedriskā transporta pieturu skaits",
                "Veikalu skaits",
                "Tirdzniecības centru skaits"
            ]
        ],
        *[
            (pl.lit(1.) * pl.col(m) / pl.col("Iedzīvotāju skaits gada sākumā") ).alias(f"{m} (uz 1 iedz.)")
            for m in [
                "Ekspluatācijā pieņemto dzīvokļu platība (tūkst. m2)",
                "Migrācijas saldo",
                "Ceļu un ielu izdevumi (investīcijas)",
                "Ceļu un ielu izdevumi (uzturēšana)",
                "Izdevumi sociālā atbalsta pasākumiem",
                "Autoceļu un pašvaldību ielu garums (asfaltbetons)",
                "Pašvaldības izdevumi: Vispārējie valdības dienesti (Uzturēšanas izdevumi)",
                "Pašvaldības izdevumi: Vispārējie valdības dienesti (Kapitālie izdevumi)",
                "Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Uzturēšanas izdevumi)",
                "Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Kapitālie izdevumi)",
                "Pašvaldības izdevumi: Ekonomiskā darbība (Uzturēšanas izdevumi)",
                "Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Uzturēšanas izdevumi)",
                "Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Kapitālie izdevumi)",
                "Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Uzturēšanas izdevumi)",
                "Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Kapitālie izdevumi)",
                "Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi)",
                "Pašvaldības izdevumi: Izglītība (Kapitālie izdevumi)",
                "Pašvaldības izdevumi: Sociālā aizsardzība (Uzturēšanas izdevumi)",
                "Pašvaldības izdevumi: Sociālā aizsardzība (Kapitālie izdevumi)",
                "Pašvaldības izdevumi: Aizsardzība (Uzturēšanas izdevumi)",
                "Pašvaldības izdevumi: Ekonomiskā darbība (Kapitālie izdevumi)",
                "Pašvaldības izdevumi: Vides aizsardzība (Uzturēšanas izdevumi)",
                "Pašvaldības izdevumi: Vides aizsardzība (Kapitālie izdevumi)",
                "Pašvaldības izdevumi: Veselība (Uzturēšanas izdevumi)",
                "Pašvaldības izdevumi: Veselība (Kapitālie izdevumi)",
                "Pašvaldības izdevumi: Aizsardzība (Kapitālie izdevumi)",
                "Pašvaldības ieņēmumi: NAUDAS SODI UN SANKCIJAS",
                "Pašvaldības ieņēmumi: Ieņēmumi no iedzīvotāju ienākuma nodokļa",
                "Pašvaldības ieņēmumi: Nekustamā īpašuma nodoklis",
                "Pašvaldības ieņēmumi: Nodokļi atsevišķām precēm un pakalpojumu veidiem",
                "Pašvaldības ieņēmumi: Nodokļi un maksājumi par tiesībām lietot atsevišķas preces",
                "Pašvaldības ieņēmumi: VALSTS (PAŠVALDĪBU) NODEVAS UN KANCELEJAS NODEVAS",
                "Pašvaldības ieņēmumi: IEŅĒMUMI NO UZŅĒMĒJDARBĪBAS UN ĪPAŠUMA",
                "Produkcijas vērtība (EUR)",
                "Pievienotā vērtība (EUR)",
            ]
        ],
        *[
            (pl.lit(1.) * pl.col(m) / pl.col("Platība (km^2)")).alias(f"{m} (uz km^2)")
            for m in [
                "Ceļu un ielu izdevumi (investīcijas)",
                "Ceļu un ielu izdevumi (uzturēšana)"
            ]
        ]
    )
    .drop(starting_columns)
    .to_pandas()
)

In [29]:
territorial_information_full = (
    territorial_information
    .merge(derived_measures, on=['Teritoriālā vienība', 'Gads'], how="left")
)




In [30]:
territorial_information_full.to_file("./data/transformed/territorial_information_summary.geojson", driver="GeoJSON")